# 01_EDA – Exploratory Data Analysis on Login Event Data

This notebook performs exploratory data analysis (EDA) on the RBA login dataset to understand user login behaviors, identify key patterns, and prepare the data for anomaly detection using machine learning.

### Objectives:
- Inspect dataset structure and types
- Analyze login distributions by country, time, and user behavior
- Visualize geo-based and temporal login trends
- Identify potential indicators of malicious activity (e.g., rare locations, odd login hours)
- Guide feature engineering for downstream ML models

**Imports**:

In [1]:
# Standart Imports
import dask.dataframe as dd
import pandas as pd
import numpy as np

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

**Load the Dataset**:

In [2]:
df = dd.read_csv('../data/rba-dataset.csv')
df.head()

,index,Login Timestamp,User ID,Round-Trip Time [ms],IP Address,Country,Region,City,ASN,User Agent String,Browser Name and Version,OS Name and Version,Device Type,Login Successful,Is Attack IP,Is Account Takeover
0,0,2020-02-03 12:43:30.772,-4324475583306591935,NaN,10.0.65.171,NO,-,-,29695,Mozilla/5.0 (iPhone; CPU iPhone OS 13_4 like ...,Firefox 20.0.0.1618,iOS 13.4,mobile,False,False,False
1,1,2020-02-03 12:43:43.549,-4324475583306591935,NaN,194.87.207.6,AU,-,-,60117,Mozilla/5.0 (Linux; Android 4.1; Galaxy Nexus...,Chrome Mobile 46.0.2490,Android 4.1,mobile,False,False,False
2,2,2020-02-03 12:43:55.873,-3284137479262433373,NaN,81.167.144.58,NO,Vestland,Urangsvag,29695,Mozilla/5.0 (iPad; CPU OS 7_1 like Mac OS X) ...,Android 2.3.3.2672,iOS 7.1,mobile,True,False,False
3,3,2020-02-03 12:43:56.180,-4324475583306591935,NaN,170.39.78.152,US,-,-,393398,Mozilla/5.0 (Linux; Android 4.1; Galaxy Nexus...,Chrome Mobile WebView 85.0.4183,Android 4.1,mobile,False,False,False
4,4,2020-02-03 12:43:59.396,-4618854071942621186,NaN,10.0.0.47,US,Virginia,Ashburn,398986,Mozilla/5.0 (Linux; U; Android 2.2) Build/NMA...,Chrome Mobile WebView 85.0.4183,Android 2.2,mobile,False,True,False


C:\Users\emrev\PycharmProjects\ML-SIEM\.venv\lib\site-packages\dask\dataframe\core.py:382: UserWarning: Insufficient elements for `head`. 20 elements requested, only 0 elements available. Try passing larger `npartitions` to `head`.
  warnings.warn(


**Basic EDA**:

In [3]:
df.info()

<class 'dask.dataframe.dask_expr.DataFrame'>
Columns: 16 entries, index to Is Account Takeover
dtypes: bool(3), float64(1), int64(3), string(9)

In [4]:
df.describe().compute()

,index,User ID,Round-Trip Time [ms],ASN
count,3.126926e+07,3.126926e+07,1.275935e+06,3.126926e+07
mean,1.563463e+07,-2.689563e+11,6.639332e+02,1.621215e+05
std,9.026659e+06,4.514276e+18,1.116125e+03,1.719185e+05
min,0.000000e+00,-9.223371e+18,8.000000e+00,1.200000e+01
25%,7.814304e+06,-4.324476e+18,4.740000e+02,2.969500e+04
50%,1.558211e+07,-4.324476e+18,5.440000e+02,2.071740e+05
75%,2.344399e+07,2.293924e+18,6.970000e+02,3.933980e+05
max,3.126926e+07,9.223359e+18,2.234570e+05,5.077270e+05


C:\Users\emrev\PycharmProjects\ML-SIEM\.venv\lib\site-packages\dask\dataframe\core.py:382: UserWarning: Insufficient elements for `head`. 20 elements requested, only 0 elements available. Try passing larger `npartitions` to `head`.
  warnings.warn(


In [5]:
df.columns

Index(['index', 'Login Timestamp', 'User ID', 'Round-Trip Time [ms]',
       'IP Address', 'Country', 'Region', 'City', 'ASN', 'User Agent String',
       'Browser Name and Version', 'OS Name and Version', 'Device Type',
       'Login Successful', 'Is Attack IP', 'Is Account Takeover'],
      dtype='object')

In [6]:
df.dtypes

index                                 int64
Login Timestamp             string[pyarrow]
User ID                               int64
Round-Trip Time [ms]                float64
IP Address                  string[pyarrow]
Country                     string[pyarrow]
Region                      string[pyarrow]
City                        string[pyarrow]
ASN                                   int64
User Agent String           string[pyarrow]
Browser Name and Version    string[pyarrow]
OS Name and Version         string[pyarrow]
Device Type                 string[pyarrow]
Login Successful                       bool
Is Attack IP                           bool
Is Account Takeover                    bool
dtype: object

In [7]:
# Check NaN
df.isna().sum().compute()

index                              0
Login Timestamp                    0
User ID                            0
Round-Trip Time [ms]        29993329
IP Address                         0
Country                            0
Region                         47409
City                            8590
ASN                                0
User Agent String                  0
Browser Name and Version           0
OS Name and Version                0
Device Type                     1526
Login Successful                   0
Is Attack IP                       0
Is Account Takeover                0
dtype: int64

In [8]:
# Length of the full dataset
len(df)

31269264

In [9]:
# Check the value count of User ID
df["User ID"].value_counts().compute().describe()

count    4.304857e+06
mean     7.263717e+00
std      6.760161e+03
min      1.000000e+00
25%      1.000000e+00
50%      2.000000e+00
75%      4.000000e+00
max      1.402590e+07
Name: count, dtype: float64

In the next steps, I will clean and transform the dataset based on early observations:

* `User ID` will be kept as there are approximently 7 rows for each user.
* `Round-Trip Time [ms]` contains too many missing values (over 95%) and will be dropped.
* `Region` and `City` are partially missing and less useful compared to `Country`, which I’ll retain as the primary geolocation feature.
* `User Agent String` will be dropped since I already have separate columns for browser, OS, and device type.
* I may extract major versions from browser or OS strings later if needed, but for now I’ll keep them as-is.
* `Is Attack IP` will be kept to help label or validate suspicious activity.
* Finally, I’ll drop the `index` column as it doesn’t serve any purpose.

For time-based analysis, I will extract only the hour from the `Login Timestamp`. Using year, month, or day likely won’t add meaningful patterns, while minute or second would be overly granular and introduce noise. Hour-level granularity should be sufficient to identify suspicious login times.

# Feature Engineering

In this part, we will clear the dataset and make it ready for further analyzing and visualization.

**Drop Columns**:

In [10]:
dropped_columns = ["index", "Round-Trip Time [ms]", "Region", "City", "User Agent String"]
df = df.drop(columns=dropped_columns)

In [11]:
df.head()

,Login Timestamp,User ID,IP Address,Country,ASN,Browser Name and Version,OS Name and Version,Device Type,Login Successful,Is Attack IP,Is Account Takeover
0,2020-02-03 12:43:30.772,-4324475583306591935,10.0.65.171,NO,29695,Firefox 20.0.0.1618,iOS 13.4,mobile,False,False,False
1,2020-02-03 12:43:43.549,-4324475583306591935,194.87.207.6,AU,60117,Chrome Mobile 46.0.2490,Android 4.1,mobile,False,False,False
2,2020-02-03 12:43:55.873,-3284137479262433373,81.167.144.58,NO,29695,Android 2.3.3.2672,iOS 7.1,mobile,True,False,False
3,2020-02-03 12:43:56.180,-4324475583306591935,170.39.78.152,US,393398,Chrome Mobile WebView 85.0.4183,Android 4.1,mobile,False,False,False
4,2020-02-03 12:43:59.396,-4618854071942621186,10.0.0.47,US,398986,Chrome Mobile WebView 85.0.4183,Android 2.2,mobile,False,True,False


In [12]:
df.columns

Index(['Login Timestamp', 'User ID', 'IP Address', 'Country', 'ASN',
       'Browser Name and Version', 'OS Name and Version', 'Device Type',
       'Login Successful', 'Is Attack IP', 'Is Account Takeover'],
      dtype='object')

**Extract Hour and Day of the Week**:

In [13]:
df['Login Timestamp'] = dd.to_datetime(df['Login Timestamp'], format='%Y-%m-%d %H:%M:%S.%f')

In [14]:
df.head()

,Login Timestamp,User ID,IP Address,Country,ASN,Browser Name and Version,OS Name and Version,Device Type,Login Successful,Is Attack IP,Is Account Takeover
0,2020-02-03 12:43:30.772,-4324475583306591935,10.0.65.171,NO,29695,Firefox 20.0.0.1618,iOS 13.4,mobile,False,False,False
1,2020-02-03 12:43:43.549,-4324475583306591935,194.87.207.6,AU,60117,Chrome Mobile 46.0.2490,Android 4.1,mobile,False,False,False
2,2020-02-03 12:43:55.873,-3284137479262433373,81.167.144.58,NO,29695,Android 2.3.3.2672,iOS 7.1,mobile,True,False,False
3,2020-02-03 12:43:56.180,-4324475583306591935,170.39.78.152,US,393398,Chrome Mobile WebView 85.0.4183,Android 4.1,mobile,False,False,False
4,2020-02-03 12:43:59.396,-4618854071942621186,10.0.0.47,US,398986,Chrome Mobile WebView 85.0.4183,Android 2.2,mobile,False,True,False


In [15]:
df['login_hours'] = df['Login Timestamp'].dt.hour
df['login_day'] = df['Login Timestamp'].dt.weekday
df.head()

,Login Timestamp,User ID,IP Address,Country,ASN,Browser Name and Version,OS Name and Version,Device Type,Login Successful,Is Attack IP,Is Account Takeover,login_hours,login_day
0,2020-02-03 12:43:30.772,-4324475583306591935,10.0.65.171,NO,29695,Firefox 20.0.0.1618,iOS 13.4,mobile,False,False,False,12,0
1,2020-02-03 12:43:43.549,-4324475583306591935,194.87.207.6,AU,60117,Chrome Mobile 46.0.2490,Android 4.1,mobile,False,False,False,12,0
2,2020-02-03 12:43:55.873,-3284137479262433373,81.167.144.58,NO,29695,Android 2.3.3.2672,iOS 7.1,mobile,True,False,False,12,0
3,2020-02-03 12:43:56.180,-4324475583306591935,170.39.78.152,US,393398,Chrome Mobile WebView 85.0.4183,Android 4.1,mobile,False,False,False,12,0
4,2020-02-03 12:43:59.396,-4618854071942621186,10.0.0.47,US,398986,Chrome Mobile WebView 85.0.4183,Android 2.2,mobile,False,True,False,12,0


**Browser Name / Version and OS Name / Version**:

In [16]:
df['browser_name'] = df['Browser Name and Version'].str.extract(r'^([^\d]*\d*\s?[A-Za-z]+)', expand=False)
df.head()

,Login Timestamp,User ID,IP Address,Country,ASN,Browser Name and Version,OS Name and Version,Device Type,Login Successful,Is Attack IP,Is Account Takeover,login_hours,login_day,browser_name
0,2020-02-03 12:43:30.772,-4324475583306591935,10.0.65.171,NO,29695,Firefox 20.0.0.1618,iOS 13.4,mobile,False,False,False,12,0,Firefox
1,2020-02-03 12:43:43.549,-4324475583306591935,194.87.207.6,AU,60117,Chrome Mobile 46.0.2490,Android 4.1,mobile,False,False,False,12,0,Chrome Mobile
2,2020-02-03 12:43:55.873,-3284137479262433373,81.167.144.58,NO,29695,Android 2.3.3.2672,iOS 7.1,mobile,True,False,False,12,0,Android
3,2020-02-03 12:43:56.180,-4324475583306591935,170.39.78.152,US,393398,Chrome Mobile WebView 85.0.4183,Android 4.1,mobile,False,False,False,12,0,Chrome Mobile WebView
4,2020-02-03 12:43:59.396,-4618854071942621186,10.0.0.47,US,398986,Chrome Mobile WebView 85.0.4183,Android 2.2,mobile,False,True,False,12,0,Chrome Mobile WebView


In [17]:
df['OS_name'] = df['OS Name and Version'].str.extract(r'^(.*?)(?:\s+\d+.*)?$', expand=False).str.strip()
df.head()

,Login Timestamp,User ID,IP Address,Country,ASN,Browser Name and Version,OS Name and Version,Device Type,Login Successful,Is Attack IP,Is Account Takeover,login_hours,login_day,browser_name,OS_name
0,2020-02-03 12:43:30.772,-4324475583306591935,10.0.65.171,NO,29695,Firefox 20.0.0.1618,iOS 13.4,mobile,False,False,False,12,0,Firefox,iOS
1,2020-02-03 12:43:43.549,-4324475583306591935,194.87.207.6,AU,60117,Chrome Mobile 46.0.2490,Android 4.1,mobile,False,False,False,12,0,Chrome Mobile,Android
2,2020-02-03 12:43:55.873,-3284137479262433373,81.167.144.58,NO,29695,Android 2.3.3.2672,iOS 7.1,mobile,True,False,False,12,0,Android,iOS
3,2020-02-03 12:43:56.180,-4324475583306591935,170.39.78.152,US,393398,Chrome Mobile WebView 85.0.4183,Android 4.1,mobile,False,False,False,12,0,Chrome Mobile WebView,Android
4,2020-02-03 12:43:59.396,-4618854071942621186,10.0.0.47,US,398986,Chrome Mobile WebView 85.0.4183,Android 2.2,mobile,False,True,False,12,0,Chrome Mobile WebView,Android


In [18]:
df[df['browser_name'].isna()][['Browser Name and Version']].head(20)

,Browser Name and Version


In [19]:
df[df['OS_name'].isna()][['OS Name and Version']].head(20)

,OS Name and Version


As we have done the feature engineering, we will drop the columns like `Login Timestamp`, `Browser Name and Version`, and so on. We will also rename the columns. E.g. `User ID` becomes `user_id`.

In [20]:
dropped_columns = ["Login Timestamp", "IP Address", "Browser Name and Version", "OS Name and Version"]
df = df.drop(columns=dropped_columns)

In [21]:
df.head()

,User ID,Country,ASN,Device Type,Login Successful,Is Attack IP,Is Account Takeover,login_hours,login_day,browser_name,OS_name
0,-4324475583306591935,NO,29695,mobile,False,False,False,12,0,Firefox,iOS
1,-4324475583306591935,AU,60117,mobile,False,False,False,12,0,Chrome Mobile,Android
2,-3284137479262433373,NO,29695,mobile,True,False,False,12,0,Android,iOS
3,-4324475583306591935,US,393398,mobile,False,False,False,12,0,Chrome Mobile WebView,Android
4,-4618854071942621186,US,398986,mobile,False,True,False,12,0,Chrome Mobile WebView,Android


**Rename Columns**:

In [22]:
# Rename Dictionary
rename_dict = {
    "User ID": "user_id",
    "Country": "country_code",
    "ASN": "asn",
    "Device Type": "device_type",
    "Login Successful": "is_login_success",
    "Is Attack IP": "is_attack_ip",
    "Is Account Takeover": "is_account_takeover",
    "OS_name": "os_name",
}

df = df.rename(columns=rename_dict)

In [23]:
df.head()

,user_id,country_code,asn,device_type,is_login_success,is_attack_ip,is_account_takeover,login_hours,login_day,browser_name,os_name
0,-4324475583306591935,NO,29695,mobile,False,False,False,12,0,Firefox,iOS
1,-4324475583306591935,AU,60117,mobile,False,False,False,12,0,Chrome Mobile,Android
2,-3284137479262433373,NO,29695,mobile,True,False,False,12,0,Android,iOS
3,-4324475583306591935,US,393398,mobile,False,False,False,12,0,Chrome Mobile WebView,Android
4,-4618854071942621186,US,398986,mobile,False,True,False,12,0,Chrome Mobile WebView,Android


**Optimize the data**:

In [24]:
df.dtypes

user_id                        float64
country_code           string[pyarrow]
asn                            float64
device_type            string[pyarrow]
is_login_success                object
is_attack_ip                    object
is_account_takeover             object
login_hours                      int32
login_day                        int32
browser_name           string[pyarrow]
os_name                string[pyarrow]
dtype: object

In [25]:
df['login_hours'].min().compute(), df['login_hours'].max().compute()

(0, 23)

In [26]:
# Check NaN
df.isna().sum().compute()

user_id                   0
country_code              0
asn                       0
device_type            1526
is_login_success          0
is_attack_ip              0
is_account_takeover       0
login_hours               0
login_day                 0
browser_name              8
os_name                   0
dtype: int64

In [27]:
# Drop NaN
df = df.dropna()

In [28]:
# Verify drop NaN
df.isna().sum().compute()

user_id                0
country_code           0
asn                    0
device_type            0
is_login_success       0
is_attack_ip           0
is_account_takeover    0
login_hours            0
login_day              0
browser_name           0
os_name                0
dtype: int64

In [29]:
# Optimize the data types

# Float64 -> Int64
df['user_id'] = df['user_id'].astype('int64')
df['asn'] = df['asn'].astype('int64')

# Object -> Bool
df['is_login_success'] = df['is_login_success'].astype('bool')
df['is_attack_ip'] = df['is_attack_ip'].astype('bool')
df['is_account_takeover'] = df['is_account_takeover'].astype('bool')

# Int32 -> Uint8
df['login_day'] = df['login_day'].astype('uint8')
df['login_hours'] = df['login_hours'].astype('uint8')

# String -> Category
df['country_code'] = df['country_code'].astype('category')

In [30]:
# Verify
df.dtypes

user_id                          int64
country_code                  category
asn                              int64
device_type            string[pyarrow]
is_login_success                  bool
is_attack_ip                      bool
is_account_takeover               bool
login_hours                      uint8
login_day                        uint8
browser_name           string[pyarrow]
os_name                string[pyarrow]
dtype: object

In [31]:
df.head()

,user_id,country_code,asn,device_type,is_login_success,is_attack_ip,is_account_takeover,login_hours,login_day,browser_name,os_name
0,-4324475583306591935,NO,29695,mobile,False,False,False,12,0,Firefox,iOS
1,-4324475583306591935,AU,60117,mobile,False,False,False,12,0,Chrome Mobile,Android
2,-3284137479262433373,NO,29695,mobile,True,False,False,12,0,Android,iOS
3,-4324475583306591935,US,393398,mobile,False,False,False,12,0,Chrome Mobile WebView,Android
4,-4618854071942621186,US,398986,mobile,False,True,False,12,0,Chrome Mobile WebView,Android


**Save the Dataset**:

In [32]:
df.to_csv('../data/processed/', index=False)

['C:\\Users\\emrev\\PycharmProjects\\ML-SIEM\\data\\processed\\000.part',
 'C:\\Users\\emrev\\PycharmProjects\\ML-SIEM\\data\\processed\\001.part',
 'C:\\Users\\emrev\\PycharmProjects\\ML-SIEM\\data\\processed\\002.part',
 'C:\\Users\\emrev\\PycharmProjects\\ML-SIEM\\data\\processed\\003.part',
 'C:\\Users\\emrev\\PycharmProjects\\ML-SIEM\\data\\processed\\004.part',
 'C:\\Users\\emrev\\PycharmProjects\\ML-SIEM\\data\\processed\\005.part',
 'C:\\Users\\emrev\\PycharmProjects\\ML-SIEM\\data\\processed\\006.part',
 'C:\\Users\\emrev\\PycharmProjects\\ML-SIEM\\data\\processed\\007.part',
 'C:\\Users\\emrev\\PycharmProjects\\ML-SIEM\\data\\processed\\008.part',
 'C:\\Users\\emrev\\PycharmProjects\\ML-SIEM\\data\\processed\\009.part',
 'C:\\Users\\emrev\\PycharmProjects\\ML-SIEM\\data\\processed\\010.part',
 'C:\\Users\\emrev\\PycharmProjects\\ML-SIEM\\data\\processed\\011.part',
 'C:\\Users\\emrev\\PycharmProjects\\ML-SIEM\\data\\processed\\012.part',
 'C:\\Users\\emrev\\PycharmProjects\\M

In this stage, we prepared the dataset for deeper analysis by performing several key operations. First, we **dropped unnecessary columns** such as raw timestamps, user agents, and original browser/OS info after extracting the relevant parts. We then **renamed columns** to follow a consistent and readable naming convention (e.g., `User ID` → `user_id`). From selected columns, we **derived new features** like `login_hours`, `login_day`, `browser_name`, and `os_name` to capture essential patterns while reducing noise. Next, we **removed rows with missing values** to maintain data quality. Finally, we **optimized data types** — converting numerical columns to appropriate integer types, booleans to native `bool`, and string columns with low cardinality to `category` — to enhance performance and reduce memory usage.